# 1 Preliminaries
## 1.1 Import
Import required libraries.

In [33]:
import os
import numpy as np
from keras import callbacks, models, layers
import pandas as pd
import chess.pgn

## 1.2 Load Data
Load the pgn file and create DataFrame. It should have three columns:
- Color
- Board
- Move

In [34]:
def read_data(file):
    res = []
    while True:
        try:
            game = chess.pgn.read_game(file)
            
            if game is None:
                break
                
            board = game.board()
            
            color = "white" if game.headers["White"] == file.name[:-4] else "black"
            
            for move in game.mainline_moves():
                if(color == "white" and board.turn == chess.WHITE) or (color == "black" and board.turn == chess.BLACK):
                    res.append({
                        "board": board.fen(),
                        "color": color,
                        "move": move,
                    })
                board.push(move)
        except Exception as e:
            print(e)
    return res

data = []
# directory = 'data'
# for filename in os.listdir(directory):
#     f = os.path.join(directory, filename)
#     
#     if os.path.isfile(f) and filename[-3:] == "pgn":
#         with open(f, 'r') as pgn:
#             print(f)
#             data.append(read_data(pgn))
      
with open('data/damnsaltythatsport.pgn', 'r') as pgn:
    data.extend(read_data(pgn))
            
df = pd.DataFrame(data)
df

,board,color,move
0,rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR ...,black,g8f6
1,rnbqkb1r/pppppppp/5n2/8/2PP4/8/PP2PPPP/RNBQKBN...,black,e7e6
2,rnbqkb1r/pppp1ppp/4pn2/8/2PP4/2N5/PP2PPPP/R1BQ...,black,f8e7
3,rnbqk2r/ppppbppp/4pn2/8/2PPP3/2N5/PP3PPP/R1BQK...,black,e8g8
4,rnbq1rk1/ppppbppp/4pn2/8/2PPP3/2NB4/PP3PPP/R1B...,black,d7d6
...,...,...,...
2669,r3k2r/pp1q1p1p/2p5/Q4p2/3b1P1P/2Np2P1/PPnB2BK/...,black,e8g8
2670,r4rk1/pp1q1p1p/2p5/Q4p2/3b1P1P/2Np2PB/PPnB3K/5...,black,b7b5
2671,r4rk1/p2q1p1p/2p5/Qp3p2/3b1P1P/2Np1RPB/PPnB3K/...,black,f8e8
2672,r3r1k1/p2q1p1p/2p5/Qp3p2/3b1P1P/2NR2PB/PPnB3K/...,black,d4g1


## 1.3 Visualize Data
Visualize the board to make sure we're chilling.


In [35]:
def fen_to_board(fen):
    """Function written by ChatGPT"""
    piece_to_char = {
        'p': 'p', 'r': 'r', 'n': 'n', 'b': 'b', 'q': 'q', 'k': 'k',
        'P': 'P', 'R': 'R', 'N': 'N', 'B': 'B', 'Q': 'Q', 'K': 'K',
        '1': '.', '2': '.', '3': '.', '4': '.', '5': '.', '6': '.', '7': '.', '8': '.'
    }
    board = []
    for row in fen.split()[0].split('/'):
        board_row = []
        for char in row:
            if char.isdigit():
                board_row.extend(['.'] * int(char))
            else:
                board_row.append(piece_to_char[char])
        board.append(board_row)
    return np.array(board)

for i in range(5):
    print(fen_to_board(df['board'][i]))
    print(df['move'][i])
    print('\n')

[['r' 'n' 'b' 'q' 'k' 'b' 'n' 'r']
 ['p' 'p' 'p' 'p' 'p' 'p' 'p' 'p']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' 'P' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['P' 'P' 'P' '.' 'P' 'P' 'P' 'P']
 ['R' 'N' 'B' 'Q' 'K' 'B' 'N' 'R']]
g8f6


[['r' 'n' 'b' 'q' 'k' 'b' '.' 'r']
 ['p' 'p' 'p' 'p' 'p' 'p' 'p' 'p']
 ['.' '.' '.' '.' '.' 'n' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' 'P' 'P' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['P' 'P' '.' '.' 'P' 'P' 'P' 'P']
 ['R' 'N' 'B' 'Q' 'K' 'B' 'N' 'R']]
e7e6


[['r' 'n' 'b' 'q' 'k' 'b' '.' 'r']
 ['p' 'p' 'p' 'p' '.' 'p' 'p' 'p']
 ['.' '.' '.' '.' 'p' 'n' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' 'P' 'P' '.' '.' '.' '.']
 ['.' '.' 'N' '.' '.' '.' '.' '.']
 ['P' 'P' '.' '.' 'P' 'P' 'P' 'P']
 ['R' '.' 'B' 'Q' 'K' 'B' 'N' 'R']]
f8e7


[['r' 'n' 'b' 'q' 'k' '.' '.' 'r']
 ['p' 'p' 'p' 'p' 'b' 'p' 'p' 'p']
 ['.' '.' '.' '.' 'p' 'n' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '

## 1.4 Clean Data
Clean data.

In [36]:
# df['color'] = df['color'].apply(lambda x: 1 if x == 'white' else 0)
# df['move'] = df['move'].astype('category')
df.dtypes

board    object
color    object
move     object
dtype: object

## 1.5 Split Data

In [37]:
df_train = df.sample(frac=0.8)
df_valid = df.drop(df_train.index)

X_train = df_train.drop('move', axis=1)
X_valid = df_valid.drop('move', axis=1)
y_train = df_train['move']
y_valid = df_valid['move']

# 2 Train Model
## 2.1 Create Model

In [41]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,  # minimium amount of change to count as an improvement
    patience=20,  # how many epochs to wait before stopping
    restore_best_weights=True,
)

model = models.Sequential([
    
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
)

## 2.2 Fit Model

In [42]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=1,
    epochs=500,
    callbacks=[early_stopping],
    verbose=0,
)

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()

model.save('deeplearningmodel.h5')

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Move).